In [1]:
import pandas as pd
import os
from scipy.optimize import curve_fit
import numpy as np

In [2]:
data_path = os.path.join(os.getcwd(), "data", "word_counts_news_clean.txt")

# Assuming your data is in a CSV file with 'word' and 'count' columns
data = pd.read_csv(data_path, names=['count', 'word'])

In [16]:
a = np.array([1,2])
np.append(a,3)

array([1, 2, 3])

In [17]:
n = 1000
sparce = 2

types_dict = dict()

for i in range(0, n, sparce):
    m = data.sample(n = i+1, weights="count",replace=True, random_state=123)
    types_dict[i+1] = len(set(m.word))

In [18]:
n_words

array([], dtype=float64)

In [9]:
types_dict

{1: 1,
 3: 3,
 5: 5,
 7: 7,
 9: 9,
 11: 11,
 13: 13,
 15: 15,
 17: 17,
 19: 19,
 21: 20,
 23: 22,
 25: 24,
 27: 26,
 29: 28,
 31: 30,
 33: 31,
 35: 32,
 37: 33,
 39: 35,
 41: 37,
 43: 38,
 45: 40,
 47: 42,
 49: 44,
 51: 46,
 53: 48,
 55: 49,
 57: 50,
 59: 51,
 61: 53,
 63: 55,
 65: 57,
 67: 59,
 69: 61,
 71: 63,
 73: 65,
 75: 67,
 77: 68,
 79: 70,
 81: 71,
 83: 72,
 85: 74,
 87: 76,
 89: 78,
 91: 80,
 93: 81,
 95: 81,
 97: 83,
 99: 85}

In [ ]:
curve_fit

What best values of (a,b,c) and (k,beta) did you find? How did you find them -
what method did you use?